<a href="https://colab.research.google.com/github/daniellorieri/Estudos_E_Projetos_MachineLearn/blob/main/census_apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Objetivo
######Mostrar o funcionamento do algoritmo Apriori que tem como objetivo fazer associação entre os dados e gerar regra de associação, baseado na lógica, se isso --> (então) aquilo. Se X --> Y.
 - Iremos fazer a associação dos atributos categóricos utilizando o algoritimo apriori e verificar o resultado.

###Dados
 - Conjunto de dados utilizados é da base de dados census.csv

###Importando Bibliotecas

In [29]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

###Lendo a base de dados

In [30]:
data = pd.read_csv('/content/drive/My Drive/dataset/census.csv')

####Dicionario de dados
- age: idade
- workclass: classe de trabalho
- final-weight: peso final
- education: Educação
- education-num: educação-num
- marital-status: Estado civil
- occupation: ocupação
- relationship
- race: relacionamento corrida
- sex
- capital-gain: ganho de capital
- capital-loos: perda capital
- hour-per-week: hora por semana
- native-country: país nativo
- income: renda por ano

###Visualizando os 5 primeiros registros





In [31]:
data.head()


,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


####Transformação dos dados

In [32]:
#corrigindo nome das colunas
data.columns = data.columns.str.replace('-', '_')

#visalizando novamente
data.head()

,age,workclass,final_weight,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loos,hour_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


###Exibindo os tipos de variáveis

In [33]:
display(data.dtypes)

age                int64
workclass         object
final_weight       int64
education         object
education_num      int64
marital_status    object
occupation        object
relationship      object
race              object
sex               object
capital_gain       int64
capital_loos       int64
hour_per_week      int64
native_country    object
income            object
dtype: object

- Variável idade é do tipo numérica, para trabalharmos com o apriori precisamos converter para string.
- Vamos dividir as idades por faixas em uma distribuição de frequencias.

In [34]:
#Separando as idades por classes(faixas)
data['age'] = pd.cut(data['age'], bins=[0, 17, 25, 40, 60, 90],
                     labels=['Faixa1', "Faixa2", "Faixa3", "Faixa4", "Faixa5"])

In [35]:
#Visualizando novamente
data.head()

,age,workclass,final_weight,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loos,hour_per_week,native_country,income
0,Faixa3,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,Faixa4,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,Faixa3,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,Faixa4,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,Faixa3,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


####Variáveis categóricas:
 - Gerar um dataset com as variáveis categóricas que iremos trabalhar com o algoritmo apriori.

In [36]:
df_apriori = data[['age','workclass','education','marital_status',
                   'occupation','relationship','race','sex','native_country','income']]

In [37]:
#Visualizando novo dataset
df_apriori.head()

,age,workclass,education,marital_status,occupation,relationship,race,sex,native_country,income
0,Faixa3,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States,<=50K
1,Faixa4,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,<=50K
2,Faixa3,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States,<=50K
3,Faixa4,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States,<=50K
4,Faixa3,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,<=50K


In [38]:
#Exbir o tamnho do dataset
data.shape

(32561, 15)

####Como o dataset te 32561, para este teste iremos pegar uma amostragemrandomica aleatoria de 1000 registros.


In [39]:
df_apriori = df_apriori.sample(1000)
df_apriori

,age,workclass,education,marital_status,occupation,relationship,race,sex,native_country,income
9665,Faixa4,Private,HS-grad,Never-married,Machine-op-inspct,Own-child,White,Female,Peru,<=50K
6176,Faixa3,Private,Bachelors,Married-civ-spouse,Sales,Husband,White,Male,?,<=50K
17976,Faixa3,Private,Bachelors,Never-married,Prof-specialty,Not-in-family,White,Female,United-States,<=50K
10110,Faixa2,?,Some-college,Never-married,?,Own-child,White,Male,United-States,<=50K
2275,Faixa2,Private,Some-college,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...
21482,Faixa3,Private,Bachelors,Never-married,Exec-managerial,Not-in-family,White,Male,United-States,>50K
6913,Faixa4,Local-gov,Assoc-acdm,Married-civ-spouse,Adm-clerical,Husband,White,Male,United-States,<=50K
5163,Faixa3,Private,Masters,Never-married,Sales,Own-child,White,Male,United-States,<=50K
31000,Faixa4,Private,Some-college,Married-civ-spouse,Transport-moving,Husband,White,Male,United-States,>50K


####Convertendo de DataFrame para formato Lista.

In [40]:
#Crio um varável vazia que receba uma lista.
transacoes = []
##Executo loop sobre os 1000 registros e insiro na lista.
for i in range(df_apriori.shape[0]):
  transacoes.append([str(df_apriori.values[i,j]) for j in range(df_apriori.shape[1])])

In [41]:
#Exibindo o tamanho da lista
len(transacoes)

1000

In [42]:
#Visualizar os 2 primeiros registros da lista
transacoes[:2]

[['Faixa4',
  ' Private',
  ' HS-grad',
  ' Never-married',
  ' Machine-op-inspct',
  ' Own-child',
  ' White',
  ' Female',
  ' Peru',
  ' <=50K'],
 ['Faixa3',
  ' Private',
  ' Bachelors',
  ' Married-civ-spouse',
  ' Sales',
  ' Husband',
  ' White',
  ' Male',
  ' ?',
  ' <=50K']]

####Instalando o Apyori

In [24]:
!pip install apyori

  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5974 sha256=fe1cdfb88f92fc5daf45db8a3f0a0d02c7033f67b988b0894d3f09172f9de767
  Stored in directory: /root/.cache/pip/wheels/cb/f6/e1/57973c631d27efd1a2f375bd6a83b2a616c4021f24aab84080
Successfully built apyori


In [43]:
#importando apriori
from apyori import apriori

In [44]:
#Armazenando em uma varável, a lsita passada por paramentro para o algoritmo apriori fazer as associações
regras = apriori(transacoes, min_suport = 0.3, min_confidnce = 0.2)             


In [45]:
#Trasnformando a variavel regras para em lista.
resultados = list(regras)


In [46]:
#Exibindo quantas regras foram geradas.
len(resultados)

665

In [47]:
#Visualizando as regras
resultados

[RelationRecord(items=frozenset({' <=50K'}), support=0.725, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({' <=50K'}), confidence=0.725, lift=1.0)]),
 RelationRecord(items=frozenset({' >50K'}), support=0.275, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({' >50K'}), confidence=0.275, lift=1.0)]),
 RelationRecord(items=frozenset({' Adm-clerical'}), support=0.12, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({' Adm-clerical'}), confidence=0.12, lift=1.0)]),
 RelationRecord(items=frozenset({' Bachelors'}), support=0.178, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({' Bachelors'}), confidence=0.178, lift=1.0)]),
 RelationRecord(items=frozenset({' Black'}), support=0.104, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({' Black'}), confidence=0.104, lift=1.0)]),
 RelationRecord(items=frozenset({' Craft-repair'}), suppo

In [ ]:

#  OrderedStatistic(items_base=frozenset({' Male', ' HS-grad'}), items_add=frozenset({' <=50K'}), confidence=0.7923728813559322, lift=1.092928112215079)])


###Analisando esse resultado podemos notar pela regra do se --> entao:
- Se é do sexo masculino e possui 2º grau, então ganha menos de 50 mil dólares por ano.
- Com a Confiança de 79%

###Conclusão
- Através de técnica de distribuição de frequência ajustamos a base de dados separando as idades por faixa e verificamos que foi possível trabalhar com regra de associação utilizando apriori.